# Meta

In [1]:
import sys
sys.path.append('../../../../') # TUNA root directory

In [2]:
%load_ext autoreload
%autoreload 2

import json
import pandas as pd
import re
from tqdm import tqdm
import random 
from collections import Counter
import csv
import os
from number_tokenizer.numtok import NumTok as nt
from collections import Counter
import pandas as pd
from IPython.display import display
import time

In [3]:
def homopercent(num1, num2):
    perc1 = (num1[-1] == '%')
    perc2 = (num2[-1] == '%')
    return perc1 == perc2

def tri_diff(num1, num2, num3):
    return num1 != num2 and num1 != num3 and num2 != num3
    
def tri_homopercent(num1, num2, num3):
    return homopercent(num1, num2) and homopercent(num1, num3) and homopercent(num2, num3)

# TAT QA
Get dataset from the TATQA github [here](https://github.com/NExTplusplus/TAT-QA/tree/master/dataset_raw) and place them in the `tatqa` folder.

In [5]:
with open('./tatqa/tatqa_dataset_train.json', 'r', encoding='utf-8') as f:
    tatqa_data = json.load(f)
with open('./tatqa/tatqa_dataset_dev.json', 'r', encoding='utf-8') as f:
    tatqa_data += json.load(f)
with open('./tatqa/tatqa_dataset_test.json', 'r', encoding='utf-8') as f:
    tatqa_data += json.load(f)
len(tatqa_data)

2757

In [6]:
with open('./tatqa/tatqa_dataset_train.json', 'r', encoding='utf-8') as f:
    tatqa_data = json.load(f)

In [7]:
for triplet in tatqa_data[30:70]:
    display(pd.DataFrame.from_dict(triplet['table']['table']))

,0,1,2,3
0,,,Year Ended,
1,,"June 30, 2019","June 24, 2018","June 25, 2017"
2,,,(in thousands),
3,Income tax expense computed at federal statuto...,"$513,780","$891,011","$634,086"
4,"State income taxes, net of federal tax benefit","(17,565)","(50,585)","(11,973)"
5,Foreign income taxed at different rates,"(260,344)","(939,808)","(352,860)"
6,Settlements and reductions in uncertain tax po...,"(31,291)","(33,367)","(144,519)"
7,Tax credits,"(71,779)","(69,301)","(37,713)"
8,"State valuation allowance, net of federal tax ...","26,742","57,302","12,070"
9,Equity-based compensation,"(7,566)","(35,875)","13,187"


,0,1,2,3
0,,2019,2018,2018-2019 Change
1,,,(in millions),
2,Interest income,$(24.8),$(26.7),$1.9
3,Interest expense,23.1,31.3,(8.2)
4,"Other (income) expense, net",29.5,1.4,28.1


,0,1,2,3,4,5
0,"(In millions, except per share amounts)",,,,,
1,Quarter Ended,September 30,December 31,March 31,June 30,Total
2,Fiscal Year 2019,,,,,
3,Revenue,"$ 29,084","$ 32,471","$ 30,571","$ 33,717","$ 125,843"
4,Gross margin,"19,179","20,048","20,401","23,305","82,933"
5,Operating income,"9,955","10,258","10,341","12,405","42,959"
6,Net income (a),"8,824","8,420","8,809","13,187","39,240"
7,Basic earnings per share,1.15,1.09,1.15,1.72,5.11
8,Diluted earnings per share (b),1.14,1.08,1.14,1.71,5.06
9,Fiscal Year 2018,,,,,


,0,1,2,3
0,,,"For the Years Ended December 31,",
1,,2019,2018,2017
2,Numerator:,,,
3,Consolidated net income,"$1,503","$1,848",$273
4,Denominator:,,,
5,Denominator for basic earnings per common shar...,767,762,754
6,Effect of dilutive stock options and awards un...,4,9,12
7,Denominator for diluted earnings per common sh...,771,771,766
8,Basic earnings per common share,$1.96,$2.43,$0.36
9,Diluted earnings per common share,$1.95,$2.40,$0.36


,0,1,2
0,,30 June 2019,30 June 2018
1,,$'000,$'000
2,Rights and licences,13,13
3,Internally generated software,"7,381","6,385"
4,Software under development,"16,284","6,509"
5,Total intangible assets,"23,678","12,907"


...

,0,1,2
0,,Accumulated Other Comprehensive Income (Loss),
1,,Years ended,
2,,"December 31, 2019","December 31, 2018"
3,,$,$
4,"Balance, beginning of the year","(12,216)","3,435"
5,,,
6,Other comprehensive income (loss) before recla...,"12,865","(19,821)"
7,Loss on cash flow hedges reclassified from acc...,,
8,Cost of revenues,279,255
9,Sales and marketing,"1,538","1,224"


,0,1,2
0,2019 Performance Period,Revenue Performance Goal(in millions),Operating Margin Performance Goal
1,Q1,$199.5,8.1%
2,Q2,$211.7,8.3%
3,Q3,$227.3,9.6%
4,Q4,$243.2,10.8%


,0,1,2
0,,2019,2018
1,Belgium,$49,$64
2,France,40,154
3,China,359,512
4,Russia,2,302
5,Germany,36,143
6,Italy,159,110
7,South Korea,-,314
8,Singapore,215,376
9,Other,525,469


,0,1,2,3
0,,,Fiscal,
1,,2019,2018,2017
2,Cost of sales,"$4,880","$4,403","$3,541"
3,Research and development,"2,990","3,247","2,973"
4,"Selling, general and administrative","28,596","25,088","23,911"
5,Income tax benefit,"(4,946)","(5,073)","(7,073)"
6,,"$31,520","$27,665","$23,352"


,0,1,2,3
0,,,"Year ended December 31,",
1,,,,Increase /
2,,2019,2018,(Decrease)
3,Net income - Insurance segment,$59.4,$165.2,$(105.8)
4,Effect of investment (gains) (1),(1.9),(5.6),3.7
5,Asset impairment expense,47.3,—,47.3
6,Gain on bargain purchase,(1.1),(115.4),114.3
7,Gain on reinsurance recaptures,—,(47.0),47.0
8,Acquisition costs,2.1,2.8,(0.7)
9,Insurance AOI,105.8,—,105.8


In [8]:
cell_cnt = 0
p_cnt = 0
p_cells = []
cells = []
for triplet in tqdm(tatqa_data):
    table = triplet['table']['table']
    for row in table:
        cells += row
        for cell in row:
            cell_cnt += 1
            if '(' in cell and ')' in cell:
                p_cnt += 1
                p_cells.append(cell)
print(p_cnt)
print(cell_cnt)

100%|██████████| 2201/2201 [00:00<00:00, 65032.32it/s]

10568
83201


In [9]:
vocab = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ',', '.', '(', ')', '%']

In [10]:
print(len(p_cells))
fp_cells = [_ for _ in p_cells if all([ch in vocab for ch in _])]
fp_cells[:100]

10568


['(5,580)',
 '(5,580)',
 '(43)',
 '(1,746)',
 '(1,789)',
 '(7,563)',
 '(618)',
 '(490)',
 '(1,108)',
 '(91)',
 '(3,170)',
 '(3,261)',
 '(81.9)',
 '(84.9)',
 '(85.6)',
 '(2.8)',
 '(2.6)',
 '(2.6)',
 '(2.8)',
 '(3.2)',
 '(3.6)',
 '(9.2)',
 '(8.6)',
 '(8.0)',
 '(0.3)',
 '(0.4)',
 '(14.8)',
 '(14.7)',
 '(14.6)',
 '(2.4)',
 '(0.1)',
 '(0.6)',
 '(0.1)',
 '(3.2)',
 '(2.0)',
 '(3.0)',
 '(2.4)',
 '(2.0)',
 '(3.0)',
 '(2.4)',
 '(4.5)%',
 '(3,085,697)',
 '(69)',
 '2018(1)',
 '(56,552)',
 '(55,849)',
 '(39,185)',
 '(37,035)',
 '(23,333)',
 '(32,987)',
 '(2,613)',
 '(457)',
 '(271)',
 '(1,583)',
 '(488)',
 '(330)',
 '(329)',
 '(15)',
 '(29)',
 '(11)',
 '(94)',
 '(244)',
 '(107)',
 '(39)',
 '(39)',
 '(2)',
 '(44)',
 '(879)',
 '(0.5)',
 '(1)',
 '(2)',
 '(5)',
 '(124.7)',
 '(4)',
 '(214.7)',
 '(51.7)',
 '(7.1)',
 '(6)',
 '(2,260)',
 '(2,741)',
 '(382,229)',
 '(50,391)',
 '(663,456)',
 '(4,264)',
 '(22,082)',
 '(20,374)',
 '(23,772)',
 '(3,398)',
 '(128,084)',
 '(139,662)',
 '(11,578)',
 '(153,193)',
 

In [11]:
nfp = [_ for _ in p_cells if _ not in fp_cells]
nfp[:100]

['Frozen Kefir (a)',
 'Income tax benefit (expense)',
 'Total other comprehensive income (loss), net of tax',
 '(Unaudited, in millions)',
 'Automotive and Discrete Group (ADG)',
 'Analog, MEMS and Sensors Group (AMS)',
 'Microcontrollers and Digital ICs Group (MDG)',
 'Net additions (losses)',
 '% of penetration(2)',
 'Continuing (loss)/profit before tax as shown in the consolidated income statement',
 'Aggregated expected income tax (credit)/expense',
 '(Recognition)/derecognition of deferred tax assets for losses in Luxembourg and Spain1',
 'Current year temporary differences (including losses) that we currently do not expect to use',
 'Expenses not deductible (income not taxable) for tax purposes',
 'Income tax expense/(credit)',
 'Expected term (in years)',
 'Net income (loss)',
 '$(7,904)',
 'Earnings (loss) per share:',
 '$(0.22)',
 '$(0.22)',
 '(In millions)',
 'Used lines of credit (1)',
 'Total available lines of credit(2)',
 '($ in millions)',
 '$(2.5)',
 'Operating income (

In [12]:
fp_cells[:1000]

['(5,580)',
 '(5,580)',
 '(43)',
 '(1,746)',
 '(1,789)',
 '(7,563)',
 '(618)',
 '(490)',
 '(1,108)',
 '(91)',
 '(3,170)',
 '(3,261)',
 '(81.9)',
 '(84.9)',
 '(85.6)',
 '(2.8)',
 '(2.6)',
 '(2.6)',
 '(2.8)',
 '(3.2)',
 '(3.6)',
 '(9.2)',
 '(8.6)',
 '(8.0)',
 '(0.3)',
 '(0.4)',
 '(14.8)',
 '(14.7)',
 '(14.6)',
 '(2.4)',
 '(0.1)',
 '(0.6)',
 '(0.1)',
 '(3.2)',
 '(2.0)',
 '(3.0)',
 '(2.4)',
 '(2.0)',
 '(3.0)',
 '(2.4)',
 '(4.5)%',
 '(3,085,697)',
 '(69)',
 '2018(1)',
 '(56,552)',
 '(55,849)',
 '(39,185)',
 '(37,035)',
 '(23,333)',
 '(32,987)',
 '(2,613)',
 '(457)',
 '(271)',
 '(1,583)',
 '(488)',
 '(330)',
 '(329)',
 '(15)',
 '(29)',
 '(11)',
 '(94)',
 '(244)',
 '(107)',
 '(39)',
 '(39)',
 '(2)',
 '(44)',
 '(879)',
 '(0.5)',
 '(1)',
 '(2)',
 '(5)',
 '(124.7)',
 '(4)',
 '(214.7)',
 '(51.7)',
 '(7.1)',
 '(6)',
 '(2,260)',
 '(2,741)',
 '(382,229)',
 '(50,391)',
 '(663,456)',
 '(4,264)',
 '(22,082)',
 '(20,374)',
 '(23,772)',
 '(3,398)',
 '(128,084)',
 '(139,662)',
 '(11,578)',
 '(153,193)',
 

In [13]:
# Get data

cells = []
paras = []
questions = []
answers = []

for triplet in tatqa_data:
    try:
        for row in triplet['table']['table']:
            cells += row
        for para in triplet['paragraphs']:
            paras.append(para['text'])
        for q in triplet['questions']:
            questions.append(q['question'])
            if 'answer' in q:
                answer = q['answer']
                if isinstance(answer, list):
                    answers += answer
                elif isinstance(answer, (float, int)):
                    answers.append(str(answer))
                elif isinstance(answer, str):
                    answers.append(answer)
    except:
        print(triplet)
tatqa_texts = cells + paras + questions + answers

print('# of All texts (cells, paragraphs, questions, answers) in TAT QA train+valid+test sets: ')
print(len(tatqa_texts))

# of All texts (cells, paragraphs, questions, answers) in TAT QA train+valid+test sets: 
122657


## Get stats

In [14]:
numtoked = [nt.find_numbers(cell) for cell in cells]

In [15]:
print('Number of cells: ', len(cells))
print('NumToked: ', len(numtoked))
print('Non-empty cells: ', len([cell for cell in cells if len(cell) > 0]))
print('Cells with one or more numbers: ', len([_ for _ in numtoked if len(_) > 0]))
print('Cells with two or more numbers: ', len([_ for _ in numtoked if len(_) > 1]))

Number of cells:  83201
NumToked:  83201
Non-empty cells:  70917
Cells with one or more numbers:  49645
Cells with two or more numbers:  2205


In [16]:
tatqa_segments = [nt._get_segments(cell) for cell in cells]
tatqa_table_segments = []
for seg in tatqa_segments:
    tatqa_table_segments += seg
prevs = [_[2] for _ in tatqa_table_segments]

cnt = Counter(prevs)
print('Ten most seen previous string in tables:')
print(sorted(cnt.items(), key=lambda x:x[1], reverse=True)[:10])

Ten most seen previous string in tables:
[(True, 35258), (False, 16785)]


## TAT QA single

In [17]:
tatqa_segments = [nt._get_segments(t) for t in tatqa_texts]
tatqa_all_segments = []
for seg in tatqa_segments:
    tatqa_all_segments += seg

len(tatqa_all_segments)

112874

In [18]:
tatqa_all_numbers = [_[0] for _ in nt._get_numbers(tatqa_all_segments)]
len(tatqa_all_numbers)

113543

In [19]:
tatqa_all_numbers_unique = list(set(tatqa_all_numbers))
len(tatqa_all_numbers_unique)

25554

In [20]:
tatqa_all_numbers_unique[:10]

['81,838',
 '12,492,433',
 '2,023',
 '434,761',
 '386,126',
 '13,093',
 '1,978',
 '5,295,100',
 '543.4',
 '5,225']

## TAT QA double 
Two numbers that can be added or subtracted should be of same shape

only use single-number cells because when multiple numbers occur, it might be ranges or dates, the latter doesn't support addition/subtraction

In [21]:
tatqa_row_numbers = []
for triplet in tqdm(tatqa_data):
    for row in triplet['table']['table']:
        text = ' '.join(row)
        row_nums = nt.find_numbers(text)
        if len(row_nums) > 1:
            tatqa_row_numbers.append([_[0] for _ in row_nums])
len(tatqa_row_numbers)

100%|██████████| 2201/2201 [00:01<00:00, 1724.53it/s]


16520

In [22]:
tatqa_number_pairs = []
while len(tatqa_number_pairs) < 100000:
    chosen_row = random.choice(tatqa_row_numbers)
    num1, num2 = random.choices(chosen_row, k=2)
    if num1 != num2 and homopercent(num1, num2):
        tatqa_number_pairs.append([num1, num2])
print(len(tatqa_number_pairs))
tatqa_number_pairs

100000


[['7,200,000', '33,988'],
 ['25,752', '15,839'],
 ['24', '558'],
 ['351,597', '362,477'],
 ['393,847', '113,592'],
 ['3,655', '4,833'],
 ['12,938', '11,594'],
 ['3.6', '60'],
 ['20.6', '19.9'],
 ['1,245', '1,315'],
 ['177', '151'],
 ['4,893', '2,484'],
 ['4,766', '1,563'],
 ['262,235', '9.8'],
 ['58,166', '10,837'],
 ['53.0', '28.0'],
 ['31', '2018'],
 ['274', '186'],
 ['18,034', '14,895'],
 ['8,921', '1,433'],
 ['0.03', '0.38'],
 ['2018', '2019'],
 ['3.9', '75,384,947'],
 ['0.38', '4'],
 ['1,536', '-36.2'],
 ['50.99', '1,906'],
 ['198,500', '248,060'],
 ['1,804', '1,563'],
 ['5.36', '5.47'],
 ['25,954', '11,734'],
 ['2019', '2018'],
 ['3.5', '3.6'],
 ['496,029', '278,728'],
 ['-18%', '-19%'],
 ['2.57', '2.79'],
 ['56,467', '27,863'],
 ['2.43', '2.48'],
 ['2018', '2019'],
 ['42', '40'],
 ['19.7', '17.9'],
 ['25,825', '9,299'],
 ['1,875', '1,774'],
 ['191,320', '153,701'],
 ['2,919', '2,975'],
 ['794', '1,889'],
 ['2019', '2018'],
 ['1', '3'],
 ['52,250', '10,824'],
 ['100', '138,826'],

## TAT QA Multi

In [23]:
tatqa_number_triplets = []
while len(tatqa_number_triplets) < 100000:
    chosen_row = random.choice(tatqa_row_numbers)
    num1, num2, num3 = random.choices(chosen_row, k=3)
    
    if tri_diff(num1, num2, num3) and tri_homopercent(num1, num2, num3):
        tatqa_number_triplets.append([num1, num2, num3])
print(len(tatqa_number_triplets))

100000


In [24]:
tatqa_number_triplets[:10]

[['52,055', '5', '3'],
 ['84.3', '1', '1,343.7'],
 ['705,134', '329,946', '1,237,374'],
 ['4', '129,908', '2'],
 ['14,902', '20,764', '24,400'],
 ['5', '4.0', '0.6'],
 ['1,051.6', '1,177.2', '1,114.0'],
 ['5', '1,404', '1,596'],
 ['0.3', '1.1', '0.8'],
 ['6,532', '2,968', '4,283']]

# Tabfact
Get tabfact dataset [here](https://github.com/wenhuchen/Table-Fact-Checking/tree/master/data) and place the `all_csv` folder under `tabfact`

In [25]:
tabfact_table_texts = []
for file_name in tqdm(os.listdir('tabfact/all_csv')):
    df = pd.read_csv(os.path.join('tabfact/all_csv', file_name), sep= '#')
    for index, row in df.iterrows():
        for cell in row:
            if not isinstance(cell, str):
                cell = str(cell)
            tabfact_table_texts.append(cell)          
len(tabfact_table_texts)
tabfact_table_texts[:10]

100%|██████████| 16573/16573 [01:41<00:00, 163.54it/s]


['robinson r - 22',
 'light utility helicopter',
 '1370 lb (635 kg)',
 '497 ft square (46.2 m square)',
 '2.6 lb / ft square (14 kg / m square)',
 'bell 206b3 jetranger',
 'turboshaft utility helicopter',
 '3200 lb (1451 kg)',
 '872 ft square (81.1 m square)',
 '3.7 lb / ft square (18 kg / m square)']

In [28]:
tabfact_statement_texts = []
for file_name in tqdm(os.listdir('tabfact/collected_data')):
    with open(os.path.join('tabfact/collected_data', file_name), 'r') as f:
        data = json.load(f)
        for _ in data.values():
            tabfact_statement_texts += _[0]
            tabfact_statement_texts += [_[2]]
len(tabfact_statement_texts)
tabfact_statement_texts[:10]

100%|██████████| 2/2 [00:00<00:00, 12.17it/s]


['the total attendance for week 8 is 48667',
 '1982 new orleans saints season',
 'the attendance in week 2 was 20114',
 'the record after the game on july 28 was 2 - 1',
 'the 8 - 4 win against the eskimos was in week 12',
 '1988 toronto argonauts season',
 'alex yoong started in grid position 22',
 'driver jacques villeneuve for bar - honda constructor was on grid 9',
 'driver pedro de la rosa had 29 laps',
 'the time / retired for the driver on grid 18 was brakes']

In [29]:
tabfact_all_texts = tabfact_table_texts + tabfact_statement_texts
len(tabfact_all_texts)

1550272

## Tabfact single

In [30]:
tabfact_segments = [nt._get_segments(t) for t in tabfact_all_texts]
tabfact_all_segments = []
for seg in tabfact_segments:
    tabfact_all_segments += seg

print(len(tabfact_all_segments))
tabfact_all_segments[:10]

1305412


[('22', 13, True),
 ('1370', 0, True),
 ('635', 9, False),
 ('497', 0, True),
 ('46.2', 15, False),
 ('2.6', 0, True),
 ('14', 20, False),
 ('206', 5, True),
 ('3', 9, False),
 ('3200', 0, True)]

In [31]:
tabfact_all_numbers = [_[0] for _ in nt._get_numbers(tabfact_all_segments)]
print(len(tabfact_all_numbers))
tabfact_all_numbers[:10]

1307806


['22', '1370', '635', '497', '46.2', '2.6', '14', '206', '3', '3200']

In [32]:
tabfact_all_numbers_unique = list(set(tabfact_all_numbers))
len(tabfact_all_numbers_unique)

64365

## TabFact double 
Two numbers that can be added or subtracted should be of same shape and of same column in TabFact

only use single-number cells because when multiple numbers occur, it might be ranges or dates, the latter doesn't support addition/subtraction

In [33]:
tabfact_col_numbers = []
for file_name in tqdm(os.listdir('tabfact/all_csv')):
    df = pd.read_csv(os.path.join('tabfact/all_csv', file_name), sep= '#')
    for header in df.columns:
        col = df[header]
        col_text = [str(cell) for cell in df[header]]
        col_numbers = [nt.find_numbers(t) for t in col_text]
        if any([len(_) != 1 for _ in col_numbers]) or len(col_numbers) == 0:
            # not one number per cell
            continue 
        col_numbers = [_[0][0] for _ in col_numbers]
        tabfact_col_numbers.append(col_numbers)
len(tabfact_col_numbers)
        

100%|██████████| 16573/16573 [02:20<00:00, 117.83it/s]


36074

In [34]:
tabfact_col_numbers[:3]

[['13', '6', '45', '34', '34'],
 ['44',
  '24',
  '12',
  '23',
  '34',
  '20',
  '17',
  '32',
  '13',
  '14',
  '42',
  '35',
  '42',
  '3',
  '3',
  '21',
  '14'],
 ['8', '21', '25', '15', '1', '13', '7', '1', '23', '54', '35', '30']]

In [35]:
tabfact_number_pairs = []
while len(tabfact_number_pairs) < 100000:
    chosen_col = random.choice(tabfact_col_numbers)
    num1, num2 = random.choices(chosen_col, k=2)
    if num1 != num2 and homopercent(num1, num2):
        tabfact_number_pairs.append([num1, num2])
print(len(tabfact_number_pairs))
tabfact_number_pairs[:10]

100000


[['38', '33'],
 ['5.0', '4.0'],
 ['4', '9'],
 ['1940', '1942'],
 ['5', '4'],
 ['0', '6'],
 ['49552', '16101'],
 ['8', '15'],
 ['24', '20'],
 ['19', '7']]

## TabFact Multi

In [36]:
tabfact_number_triplets = []
tic = time.time()
while len(tabfact_number_triplets) < 100000:
    
    chosen_col = random.choice(tabfact_col_numbers)
    num1, num2, num3 = random.choices(chosen_col, k=3)
    
    if tri_diff(num1, num2, num3) and tri_homopercent(num1, num2, num3):
        tabfact_number_triplets.append([num1, num2, num3])
print(len(tabfact_number_triplets))

100000


In [37]:
tabfact_number_triplets[:10]

[['16', '18', '2'],
 ['26', '6', '20'],
 ['0', '12', '10'],
 ['245', '477', '193'],
 ['17486', '17787', '14964'],
 ['202', '214', '225'],
 ['2000', '1997', '1985'],
 ['16', '2', '10'],
 ['51494', '16922', '9162'],
 ['6.69', '5.27', '5.02']]

# WTQ

In [39]:
query_dev_path = 'wtq/random-split-1-dev.tsv'
query_train_path = 'wtq/random-split-1-train.tsv'

# wtq_all_queris = dev + train
wtq_all_query_texts = []
for file_name in tqdm([query_dev_path, query_train_path]):
    with open(file_name, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            row_joined = ''.join(row)
            if len(row_joined.split('\t')) != 4:
                continue
            wtq_all_query_texts.append(row_joined.split('\t')[1])
len(wtq_all_query_texts)

100%|██████████| 2/2 [00:00<00:00, 52.18it/s]


14058

In [40]:
wtq_all_query_texts[:10]

['utterance',
 'which team won previous to crettyard?',
 'how many more passengers flew to los angeles than to saskatoon from manzanillo airport in 2013?',
 'after winning on four credits with a full house what is your payout?',
 'which players played the same position as ardo kreek?',
 'which athlete was from south korea after the year 2010?',
 'what was the venue when he placed first?',
 'who ranked right after turkey?',
 'what is the total population in dzhebariki-khaya?',
 'who was the top ranked competitor in this race?']

In [41]:
wtq_csv_dirs = next(os.walk('wtq'))[1]
wtq_tsv_files = []
for dir_path in wtq_csv_dirs:
    files = os.listdir(os.path.join('wtq', dir_path))
    wtq_tsv_files += [os.path.join('wtq', dir_path, _) for _ in files if _.endswith('.tsv')]
print(len(wtq_tsv_files))

2108


In [42]:
wtq_all_table_texts = []
for file_path in tqdm(wtq_tsv_files):
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            row_joined = ' '.join(row)
            wtq_all_table_texts += row_joined.split('\t')
len(wtq_all_table_texts)

100%|██████████| 2108/2108 [00:01<00:00, 1432.55it/s]


378529

In [43]:
wtq_all_table_texts[:10]

['Year',
 'Title',
 'Chart-Positions\\nUK',
 'Chart-Positions\\nUS',
 'Chart-Positions\\nNL',
 'Comments',
 '1969',
 'Renaissance',
 '60',
 '–']

In [44]:
wtq_all_texts = wtq_all_table_texts + wtq_all_query_texts
len(wtq_all_texts)

392587

In [45]:
wtq_all_texts[:10]

['Year',
 'Title',
 'Chart-Positions\\nUK',
 'Chart-Positions\\nUS',
 'Chart-Positions\\nNL',
 'Comments',
 '1969',
 'Renaissance',
 '60',
 '–']

## WTQ single

In [46]:
wtq_segments = [nt._get_segments(t) for t in wtq_all_texts]
wtq_all_segments = []
for seg in wtq_segments:
    wtq_all_segments += seg

print(len(wtq_all_segments))
wtq_all_segments[:10]

273978


[('1969', 0, True),
 ('60', 0, True),
 ('10', 0, True),
 ('1971', 0, True),
 ('1976', 0, True),
 ('1972', 0, True),
 ('1973', 0, True),
 ('171', 0, True),
 ('1974', 0, True),
 ('94', 0, True)]

In [47]:
wtq_all_numbers = [_[0] for _ in nt._get_numbers(wtq_all_segments)]
print(len(wtq_all_numbers))
wtq_all_numbers[:10]

282846


['1969', '60', '10', '1971', '1976', '1972', '1973', '171', '1974', '94']

In [48]:
wtq_all_numbers_unique = list(set(wtq_all_numbers))
len(wtq_all_numbers_unique)

20805

## WTQ double

In [49]:
wtq_col_numbers = []
for file_path in wtq_tsv_files:
    df = pd.read_csv(file_path, sep= '\t', error_bad_lines=False, encoding='utf-8', engine='python')
    for header in df.columns:
        col = df[header]
        col_text = [str(cell) for cell in df[header]]
        col_numbers = [nt.find_numbers(t) for t in col_text]
        if any([len(_) != 1 for _ in col_numbers]) or len(col_numbers) == 0:
            # not one number per cell
            continue 
        for num in col_numbers:
            if num[0][0] == '':
                print(col_numbers)
        col_numbers = [_[0][0] for _ in col_numbers]
        wtq_col_numbers.append(col_numbers)
len(wtq_col_numbers)

Skipping line 4: '	' expected after '"'
Skipping line 5: '	' expected after '"'
Skipping line 6: '	' expected after '"'
Skipping line 8: '	' expected after '"'
Skipping line 10: '	' expected after '"'
Skipping line 16: '	' expected after '"'
Skipping line 3: '	' expected after '"'
Skipping line 4: '	' expected after '"'
Skipping line 9: '	' expected after '"'
Skipping line 20: '	' expected after '"'
Skipping line 22: '	' expected after '"'
Skipping line 23: '	' expected after '"'
Skipping line 2: '	' expected after '"'
Skipping line 8: '	' expected after '"'
Skipping line 9: '	' expected after '"'
Skipping line 10: '	' expected after '"'
Skipping line 14: '	' expected after '"'
Skipping line 17: '	' expected after '"'
Skipping line 20: '	' expected after '"'
Skipping line 5: '	' expected after '"'
Skipping line 8: '	' expected after '"'
Skipping line 14: '	' expected after '"'
Skipping line 7: '	' expected after '"'
Skipping line 11: '	' expected after '"'
Skipping line 3: '	' expected

Skipping line 2: '	' expected after '"'
Skipping line 3: '	' expected after '"'
Skipping line 4: '	' expected after '"'
Skipping line 5: '	' expected after '"'
Skipping line 6: '	' expected after '"'
Skipping line 7: '	' expected after '"'
Skipping line 8: '	' expected after '"'
Skipping line 9: '	' expected after '"'
Skipping line 10: '	' expected after '"'
Skipping line 11: '	' expected after '"'
Skipping line 12: '	' expected after '"'
Skipping line 13: '	' expected after '"'
Skipping line 14: '	' expected after '"'
Skipping line 15: '	' expected after '"'
Skipping line 16: '	' expected after '"'
Skipping line 17: '	' expected after '"'
Skipping line 18: '	' expected after '"'
Skipping line 19: '	' expected after '"'
Skipping line 20: '	' expected after '"'
Skipping line 21: '	' expected after '"'
Skipping line 22: '	' expected after '"'
Skipping line 23: '	' expected after '"'
Skipping line 24: '	' expected after '"'
Skipping line 25: '	' expected after '"'
Skipping line 26: '	' ex

3648

In [50]:
wtq_number_pairs = []
while len(wtq_number_pairs) < 100000:
    chosen_col = random.choice(wtq_col_numbers)
    num1, num2 = random.choices(chosen_col, k=2)
    if num1 != num2 and homopercent(num1, num2):
        wtq_number_pairs.append([num1, num2])
print(len(wtq_number_pairs))
wtq_number_pairs[:10]

100000


[['7', '32'],
 ['22', '62'],
 ['63', '66'],
 ['3', '2'],
 ['15', '28'],
 ['2', '4'],
 ['24.0', '11.0'],
 ['12', '11'],
 ['1914', '1898'],
 ['2009', '2007']]

## WTQ Multi

In [51]:
wtq_number_triplets = []
while len(wtq_number_triplets) < 100000:
    
    chosen_col = random.choice(wtq_col_numbers)
    num1, num2, num3 = random.choices(chosen_col, k=3)
    
    if tri_diff(num1, num2, num3) and tri_homopercent(num1, num2, num3):
        wtq_number_triplets.append([num1, num2, num3])
print(len(wtq_number_triplets))

100000


In [52]:
wtq_number_triplets[:10]

[['700,000', '5,200,000', '2,000,000'],
 ['40551', '794020', '001378772'],
 ['14', '28', '23'],
 ['11,417', '10,767', '11,754'],
 ['2001', '2010', '2004'],
 ['2004', '2008', '2002'],
 ['2014', '2009', '1999'],
 ['12', '3', '4'],
 ['1981', '1998', '1992'],
 ['71', '6', '8']]

# All datasets

In [53]:
print(len(tatqa_all_numbers))
tatqa_sampled_numbers = random.choices(tatqa_all_numbers, k=100000)

113543


In [54]:
print(len(tabfact_all_numbers))
tabfact_sampled_numbers = random.choices(tabfact_all_numbers, k=100000)

1307806


In [56]:
print(len(wtq_all_numbers))
wtq_all_numbers = [_ for _ in wtq_all_numbers if len(_) < 12]
wtq_sampled_numbers = random.choices(wtq_all_numbers, k=100000)

282846


## Single Number

In [57]:
sampled_numbers = tatqa_sampled_numbers + tabfact_sampled_numbers + wtq_sampled_numbers
len(sampled_numbers)

300000

In [58]:
sampled_numbers[:10]

['2018',
 '205,471',
 '31',
 '2019',
 '68325.5',
 '-35.96',
 '73,743',
 '5',
 '2017',
 '11.2']

## Number Pairs

In [59]:
number_pairs = tatqa_number_pairs + tabfact_number_pairs + wtq_number_pairs
len(number_pairs)

300000

In [60]:
number_pairs[:10]

[['7,200,000', '33,988'],
 ['25,752', '15,839'],
 ['24', '558'],
 ['351,597', '362,477'],
 ['393,847', '113,592'],
 ['3,655', '4,833'],
 ['12,938', '11,594'],
 ['3.6', '60'],
 ['20.6', '19.9'],
 ['1,245', '1,315']]

## Number Triplets

In [61]:
number_triplets = tatqa_number_triplets + tabfact_number_triplets + wtq_number_triplets
len(number_triplets)

300000

In [62]:
number_triplets[:10]

[['52,055', '5', '3'],
 ['84.3', '1', '1,343.7'],
 ['705,134', '329,946', '1,237,374'],
 ['4', '129,908', '2'],
 ['14,902', '20,764', '24,400'],
 ['5', '4.0', '0.6'],
 ['1,051.6', '1,177.2', '1,114.0'],
 ['5', '1,404', '1,596'],
 ['0.3', '1.1', '0.8'],
 ['6,532', '2,968', '4,283']]

## Dump data

In [63]:
with open('single_train.json', 'w') as f:
    json.dump(sampled_numbers[:int(0.8*len(sampled_numbers))], f)
with open('single_test.json', 'w') as f:
    json.dump(sampled_numbers[int(0.8*len(sampled_numbers)):], f)

In [64]:
with open('double_train.json', 'w') as f:
    json.dump(number_pairs[:int(0.8*len(number_pairs))], f)
with open('double_test.json', 'w') as f:
    json.dump(number_pairs[int(0.8*len(number_pairs)):], f)

In [65]:
with open('multi_train.json', 'w') as f:
    json.dump(number_triplets[:int(0.8*len(number_triplets))], f)
with open('multi_test.json', 'w') as f:
    json.dump(number_triplets[int(0.8*len(number_triplets)):], f)